In [14]:
import pandas as pd
import morph_kgc as mgc
import owlready2
from io import BytesIO

# Base Graph Load

In [15]:
config = """
[manager_mapping]
mappings = ../vocab/manager_mapping_metadata.ttl

[items_mapping]
mappings = ../vocab/items_mapping_metadata.ttl

[rep_mapping]
mappings = ../vocab/sales_rep_metadata.ttl
"""

In [16]:
g = mgc.materialize(config)

INFO | 2025-04-11 22:40:16,996 | Parallelization is not supported for win32 when running as a library. If you need to speed up your data integration pipeline, please run through the command line.
INFO | 2025-04-11 22:40:17,769 | 16 mapping rules retrieved.
INFO | 2025-04-11 22:40:17,776 | Mapping partition with 16 groups generated.
INFO | 2025-04-11 22:40:17,777 | Maximum number of rules within mapping group: 1.
INFO | 2025-04-11 22:40:17,777 | Mappings processed in 0.780 seconds.
INFO | 2025-04-11 22:40:18,030 | Number of triples generated in total: 59952.


In [17]:
# Validate the graph loaded (no inferences as we haven't run a rules engine)
query = """
prefix xsd: <http://www.w3.org/2001/XMLSchema#>
prefix : <local://security.permissions.net/vocab#>
prefix data: <local://security.permissions.net/entities#>

SELECT * WHERE {
    ?x a :Record ;
        :item ?item ;
        :country ?country ;
}
LIMIT 5
      """

query_result = g.query(query)

for row in query_result:
    print(row['x'], row['item'], row['country'])

local://security.permissions.net/entities#787689#United%20Kingdom local://security.permissions.net/entities#787689 local://security.permissions.net/entities#United%20Kingdom
local://security.permissions.net/entities#464079#France local://security.permissions.net/entities#464079 local://security.permissions.net/entities#France
local://security.permissions.net/entities#477330#Channel%20Islands local://security.permissions.net/entities#477330 local://security.permissions.net/entities#Channel%20Islands
local://security.permissions.net/entities#481005#United%20Kingdom local://security.permissions.net/entities#481005 local://security.permissions.net/entities#United%20Kingdom
local://security.permissions.net/entities#1734180#Finland local://security.permissions.net/entities#1734180 local://security.permissions.net/entities#Finland


In [18]:
g.parse("../vocab/permissions_ontology.ttl", format="turtle")

<Graph identifier=Ncbcf1d66161e457cb5b306cb8238f464 (<class 'rdflib.graph.Graph'>)>

In [23]:
# Serialize the RDFLib graph to RDF/XML format
rdf_xml_data = g.serialize(format='xml', destination='../vocab/graph_data.rdf')

In [25]:
# Load the serialized data into owlready2
onto = owlready2.get_ontology('../vocab/graph_data.rdf').load()

# Perform reasoning
owlready2.sync_reasoner()

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp c:\repos\rdf_permissions_example\.venv\Lib\site-packages\owlready2\hermit;c:\repos\rdf_permissions_example\.venv\Lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/egjfo/AppData/Local/Temp/tmpua45ynpj
* Owlready2 * HermiT took 2.388061285018921 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [27]:
# Save the ontology with inferred knowledge to an RDF file
onto.save(file="../vocab/permissions_inferred.rdf", format="rdfxml")